In [ ]:
#from cv23_lab2_2_utils import read_video, show_detection, orientation_histogram, bag_of_words, svm_train_test
from os import listdir
from os.path import isfile, join
import numpy as np
import scipy
from scipy import signal
from scipy import ndimage as nd
import cv2
from scipy.ndimage import convolve1d
import cv23_lab2_2_utils as utils
from cv23_lab2_2_utils import read_video, show_detection, orientation_histogram, bag_of_words, svm_train_test

import cv23_lab2_2_utils as utils
import cv2
import numpy as np
import matplotlib.pyplot as plt
from numpy import save


In [ ]:
import os
walking_dir = './cv23_lab2_material/part2 - SpatioTemporal/walking/'
running_dir = './cv23_lab2_material/part2 - SpatioTemporal/running/'
handwaving_dir = './cv23_lab2_material/part2 - SpatioTemporal/handwaving/'

walk_videos = [os.path.join(walking_dir, filename) for filename in os.listdir(walking_dir)]
run_videos = [os.path.join(running_dir, filename) for filename in os.listdir(running_dir)]
hand_videos = [os.path.join(handwaving_dir, filename) for filename in os.listdir(handwaving_dir)]


In [ ]:

walk_video1 = read_video(walk_videos[1], 200)
run_video1 = read_video(run_videos[1], 200)
wave_video1 = read_video(hand_videos[1], 200)



In [ ]:
Out = "processed_video1"
delta = 4
theta = 1.5
alpha = 2
kappa = 0.005
num_frames = 600

num_kernel = int(2 * np.ceil(3 * delta) + 1)  # kernel size
Gauss_1D = cv2.getGaussianKernel(num_kernel, delta)
Gauss_1D = Gauss_1D.reshape(Gauss_1D.shape[0])
Out_2 = convolve1d(Out, Gauss_1D, axis=0)

Out_3 = convolve1d(Out_2, Gauss_1D, axis=1)

num_kernel = int(2 * np.ceil(3 * theta) + 1)  # kernel size
Gauss_t = cv2.getGaussianKernel(num_kernel, theta)
Gauss_t = Gauss_t.reshape(Gauss_t.shape[0])
Out_4 = convolve1d(Out_3, Gauss_t, axis=2)


In [ ]:
def space_time_Gaus(input_video, sigma_val, tau_val):
    
    num_kernel = int(2 * np.ceil(3 * sigma_val) + 1)  # kernel size
    gauss_1d = cv2.getGaussianKernel(num_kernel, sigma_val)
    gauss_2d = gauss_1d @ gauss_1d.transpose()
    
    # Reshaping the 2D Gaussian kernel to make it 3D
    gauss_2d = gauss_2d.reshape(gauss_2d.shape[0], gauss_2d.shape[1], 1)
    
    # Creating a temporal Gaussian kernel
    num_kernel = int(2 * np.ceil(3 * tau_val) + 1)  # kernel size
    gauss_time = cv2.getGaussianKernel(num_kernel, tau_val)
    # Reshaping the temporal Gaussian kernel to make it 3D
    gauss_time = gauss_time.reshape(1, 1, gauss_time.shape[0])
    
    # Combining the three kernels
    gauss_combined = nd.convolve(gauss_time, gauss_2d)
    result = nd.convolve(input_video, gauss_combined)
    
    return result

def space_Gaus(input_video, sigma_val):
    
    num_kernel = int(2 * np.ceil(3 * sigma_val) + 1)  # kernel size
    gauss_1d = cv2.getGaussianKernel(num_kernel, sigma_val)
    gauss_2d = gauss_1d @ gauss_1d.transpose()
    
    # Reshaping the 2D Gaussian kernel to convolve with the video only in space (x, y)
    gauss_2d = gauss_2d.reshape(gauss_2d.shape[0], gauss_2d.shape[1], 1)
    
    result = nd.convolve(input_video, gauss_2d)
    
    return result


In [ ]:
def Harris_Detector(input_data, parameter1, parameter2, parameter3, parameter4):
    
    input_data = input_data.astype(np.float32)
    
    L = transform_data(input_data, parameter1, parameter2)
    
    # Using a different table to calculate derivatives in one axis
    dx = np.array([[[-1], [0], [1]]])
    Lx = nd.convolve(L, dx)
    # Changing the orientation of the table
    dy = np.array([[[-1]], [[0]], [[1]]])
    Ly = nd.convolve(L, dy)
    # Changing the orientation of the table
    dt = np.array([[[-1, 0, 1]]])
    Lt = nd.convolve(L, dt)
    
    L = np.array([[Lx * Lx, Lx * Ly, Lx * Lt], [Lx * Ly, Ly * Ly, Ly * Lt], [Lx * Lt, Ly * Lt, Lt * Lt]])
    
    # Initializing the M table
    M = L
    
    for i in range(3):
        for j in range(3):
            M[i, j] = transform_data(L[i, j], parameter1 * parameter3, parameter2 * parameter3)
    
    trace = np.trace(M)
    det = M[0, 0] * (M[1, 1] * M[2, 2] - M[1, 2] * M[2, 1]) - M[0, 1] * (M[1, 0] * M[2, 2] - M[1, 2] * M[2, 0]) \
        + M[0, 2] * (M[1, 0] * M[2, 1] - M[1, 1] * M[2, 0])
    H = det - parameter4 * trace**3
    
    return H


In [ ]:
def Gabor_Detector(input_data, parameter1, parameter2):
    
    input_data = input_data.astype(np.float32)
    
    L = modified_space_Gaus(input_data, parameter1)
    
    w = 4 / parameter2
    t = np.arange(-2 * parameter2, 2 * parameter2 + 1)
    
    h_ev = np.cos(2 * np.pi * t * w) * np.exp(-t ** 2 / (2 * parameter2 ** 2))
    h_od = np.sin(2 * np.pi * t * w) * np.exp(-t ** 2 / (2 * parameter2 ** 2))
    
    # Normalization with L1 norm
    h_ev = h_ev / np.linalg.norm(h_ev, ord=1)
    h_od = h_od / np.linalg.norm(h_od, ord=1)
    
    H = convolve1d(L, h_ev) ** 2 + convolve1d(L, h_od) ** 2
    
    return H


In [ ]:
def find_keypoints(H, num_points, sigma_val):
    
    max_points = np.dstack(np.unravel_index(np.argsort(H.ravel()), (H.shape[0], H.shape[1], H.shape[2])))
    max_points = max_points.reshape(max_points.shape[1], max_points.shape[2])
    max_points = max_points[-num_points:]
    
    keypoints = np.array([np.append(pair[:2][::-1], [pair[2], sigma_val]) for pair in max_points])
        
    return keypoints


In [ ]:
def Harris_points(input_data, sigma_val, tau_val, s_val, k_val, num_points):
    H = Harris_Detector(input_data, sigma_val, tau_val, s_val, k_val)
    keypoints = find_keypoints(H, num_points, sigma_val)
    return keypoints

def Gabor_points(input_data, sigma_val, tau_val, num_points):
    H = Gabor_Detector(input_data, sigma_val, tau_val)
    keypoints = find_keypoints(H, num_points, sigma_val)
    return keypoints


In [ ]:


# Input data
In_walk = walk_video1
In_wave = wave_video1
In_run = run_video1

# Parameters
sigma = 4
tau = 1.5
s = 2
k = 0.005
N = 600

# Harris Detector for walk video
H_Harris_walk = Harris_Detector(In_walk, sigma, tau, s, k)
# Gabor Detector for walk video
H_Gabor_walk = Gabor_Detector(In_walk, sigma, tau)

# Harris Detector for wave video
H_Harris_wave = Harris_Detector(In_wave, sigma, tau, s, k)
# Gabor Detector for wave video
H_Gabor_wave = Gabor_Detector(In_wave, sigma, tau)

# Harris Detector for run video
H_Harris_run = Harris_Detector(In_run, sigma, tau, s, k)
# Gabor Detector for run video
H_Gabor_run = Gabor_Detector(In_run, sigma, tau)

# Exporting data
np.save('H_Harris_walk.npy', H_Harris_walk)
np.save('H_Gabor_walk.npy', H_Gabor_walk)
np.save('H_Harris_wave.npy', H_Harris_wave)
np.save('H_Gabor_wave.npy', H_Gabor_wave)
np.save('H_Harris_run.npy', H_Harris_run)
np.save('H_Gabor_run.npy', H_Gabor_run)


In [ ]:

# Parameters
sigma = 4
tau = 1.5
s = 2
k = 0.005
N = 600



# Harris Detector on walk video
In = walk_video1
keypoints_harris_walk = modified_Harris_points(In, sigma, tau, s, k, N)
show_detection(In, keypoints_harris_walk, save_path='cv23_lab2_material/walk_harris')

# Harris Detector on run video
In = run_video1
keypoints_harris_run = modified_Harris_points(In, sigma, tau, s, k, N)
show_detection(In, keypoints_harris_run, save_path='cv23_lab2_material/run_harris')

# Harris Detector on wave video
In = wave_video1
keypoints_harris_wave = modified_Harris_points(In, sigma, tau, s, k, N)
show_detection(In, keypoints_harris_wave, save_path='cv23_lab2_material/wave_harris')

# Gabor Detector on walk video
In = walk_video1
keypoints_gabor_walk = modified_Gabor_points(In, sigma, tau, N)
show_detection(In, keypoints_gabor_walk, save_path='cv23_lab2_material/walk_gabor')

# Gabor Detector on run video
In = run_video1
keypoints_gabor_run = modified_Gabor_points(In, sigma, tau, N)
show_detection(In, keypoints_gabor_run, save_path='cv23_lab2_material/run_gabor')

# Gabor Detector on wave video
In = wave_video1
keypoints_gabor_wave = modified_Gabor_points(In, sigma, tau, N)
show_detection(In, keypoints_gabor_wave, save_path='cv23_lab2_material/wave_hof')


In [ ]:
def compute_TVL1(previous_frame, current_frame):
    optical_flow = cv2.DualTVL1OpticalFlow_create(nscales=1)
    flow = optical_flow.calc(previous_frame, current_frame, None)
    return flow


In [ ]:
def optical_flow_TVL1(input_frames):
    
    input_frames = input_frames.astype(np.uint8)
    
    # Initializing np arrays for flow in x and y
    flow_x = np.empty((input_frames.shape[0], input_frames.shape[1], input_frames.shape[2]))
    flow_y = np.empty((input_frames.shape[0], input_frames.shape[1], input_frames.shape[2]))
    
    # Converting to uint8 type 
    
    for t in range(input_frames.shape[2]):
        
        # If border
        if t == input_frames.shape[2] - 1:
            t = t - 1
        
        prev_frame = input_frames[:,:,t]
        current_frame = input_frames[:,:,t+1]
        
        flow = compute_TVL1(prev_frame, current_frame)
        
        flow_x[:, :, t] = flow[:, :, 1]
        flow_y[:, :, t] = flow[:, :, 0]
        
    return (flow_x, flow_y)


In [ ]:
def HOG(input_frames, interest_points, nbins=9, n=3, m=3):
    
    dy, dx, _ = np.gradient(input_frames)
    
    max_y = input_frames.shape[0]
    max_x = input_frames.shape[1]
    Y = []
    
    for point in interest_points:
        
        # Separating point's coordinates
        x = point[0]
        y = point[1]
        t = point[2]
        sigma = point[3]
        
        # Region 4*sigma around each interest point
        side = int(np.round(4 * sigma))
        
        # Using ifs to avoid going out of image dimensions
        if x - side <= 0:
            x_left = 0
        else:
            x_left = x - side
            
        if x + side + 1 >= max_x:
            x_right = max_x
        else:
            x_right = x + side + 1
        
        if y - side <= 0:
            y_up = 0
        else:
            y_up = y - side
            
        if y + side + 1 >= max_y:
            y_down = max_y
        else:
            y_down = y + side + 1
    
        
        Gx = dx[y_up:y_down, x_left:x_right, t]
        Gy = dy[y_up:y_down, x_left:x_right, t]
        
        Y.append(orientation_histogram(Gx, Gy, nbins, np.array([n,m])))
    
    Y = np.array(Y)
    return Y


In [2]:
def HOF(input_frames, interest_points, nbins=9, n=3, m=3):
    
    dx, dy = optical_flow_TVL1(input_frames)
    
    max_y = input_frames.shape[0]
    max_x = input_frames.shape[1]
    Y = []
    
    for point in interest_points:
        
        # Separating point's coordinates
        x = point[0]
        y = point[1]
        t = point[2]
        sigma = point[3]
        
        # Region 4*sigma around each interest point
        side = int(np.round(4 * sigma))
        
        # Using ifs to avoid going out of image dimensions
        if x - side <= 0:
            x_left = 0
        else:
            x_left = x - side
            
        if x + side + 1 >= max_x:
            x_right = max_x
        else:
            x_right = x + side + 1
        
        if y - side <= 0:
            y_up = 0
        else:
            y_up = y - side
            
        if y + side + 1 >= max_y:
            y_down = max_y
        else:
            y_down = y + side + 1
        
        Gx = dx[y_up:y_down, x_left:x_right, t]
        Gy = dy[y_up:y_down, x_left:x_right, t]
        
        Y.append(orientation_histogram(Gx, Gy, nbins, np.array([n,m])))
    
    Y = np.array(Y)
    return Y


In [ ]:
def HOF_HOG(In, interest_points, nbins=9, n=3, m=3):
    hof = HOF(In, interest_points, nbins=nbins, n=n, m=m)
    hog = HOG(In, interest_points, nbins=nbins, n=n, m=m)
    return np.concatenate((hog, hof))


In [ ]:
In = walk_video1
sigma = 4
tau = 1.5
N = 600

points = Gabor_points(In, sigma, tau, N)


In [ ]:


training_videos_txt = open('cv23_lab2_material/part2 - SpatioTemporal/training_videos.txt', 'r')
training_videos_names = [line.strip() for line in training_videos_txt.readlines()]

training_videos = []
test_videos = []
training_labels = []
test_labels = []
test_videos_names = []

video_directory = 'cv23_lab2_material/part2 - SpatioTemporal/'

walking_directory = os.path.join(video_directory, 'walking')
handwaving_directory = os.path.join(video_directory, 'handwaving')
running_directory = os.path.join(video_directory, 'running')

# Load walking videos
for video_name in os.listdir(walking_directory):
    video_path = os.path.join(walking_directory, video_name)
    if video_name in training_videos_names:
        training_videos.append(read_video(video_path, 200))
        training_labels.append('walk')
    else:
        test_videos.append(read_video(video_path, 200))
        test_labels.append('walk')
        test_videos_names.append(video_name)

# Load handwaving videos
for video_name in os.listdir(handwaving_directory):
    video_path = os.path.join(handwaving_directory, video_name)
    if video_name in training_videos_names:
        training_videos.append(read_video(video_path, 200))
        training_labels.append('handwaving')
    else:
        test_videos.append(read_video(video_path, 200))
        test_labels.append('handwaving')
        test_videos_names.append(video_name)

# Load running videos
for video_name in os.listdir(running_directory):
    video_path = os.path.join(running_directory, video_name)
    if video_name in training_videos_names:
        training_videos.append(read_video(video_path, 200))
        training_labels.append('run')
    else:
        test_videos.append(read_video(video_path, 200))
        test_labels.append('run')
        test_videos_names.append(video_name)

training_videos = np.array(training_videos)
test_videos = np.array(test_videos)



In [ ]:
interest_Harris_points_training = []
desc_HOG_Harris_train = []
sigma = 4
tau = 1.5
s = 2
k = 0.005
N = 600
n = int(np.round(4 * sigma))
m = int(np.round(4 * sigma))

for Input in training_videos:
    interest_points = Harris_points(Input, sigma, tau, s, k, N)
    interest_Harris_points_training.append(interest_points)
    desc = HOG(Input, interest_points, 9, n, m)
    desc_HOG_Harris_train.append(desc)


In [ ]:
interest_Harris_points_test = []
desc_HOG_Harris_test = []

for Input in test_videos:
    interest_points = Harris_points(Input, sigma, tau, s, k, N)
    interest_Harris_points_test.append(interest_points)
    desc = HOG(Input, interest_points, 9, n, m)
    desc_HOG_Harris_test.append(desc)


In [ ]:
bow_train_HOG_Harris, bow_test_HOG_Harris = bag_of_words(desc_HOG_Harris_train, desc_HOG_Harris_test, 400)

In [ ]:
i = 0
desc_HOF_Harris_test = []
for Input in test_videos:
    desc = HOF(Input, interest_Harris_points_test[i], 9, n, m)
    desc_HOF_Harris_test.append(desc)
    i += 1


In [ ]:
#Bag of Words for Harris/HOF
bow_train_HOF_Harris, bow_test_HOF_Harris = bag_of_words(desc_HOF_Harris_train, desc_HOF_Harris_test, 400)



In [ ]:
# Harris/HOF_HOG descriptors for training videos
desc_HOF_HOG_Harris_train = []
for i, Input in enumerate(training_videos):
    desc = HOF_HOG(Input, interest_Harris_points_training[i], 9, n, m)
    desc_HOF_HOG_Harris_train.append(desc)

# Harris/HOF_HOG descriptors for test videos
desc_HOF_HOG_Harris_test = []
for i, Input in enumerate(test_videos):
    desc = HOF_HOG(Input, interest_Harris_points_test[i], 9, n, m)
    desc_HOF_HOG_Harris_test.append(desc)


In [ ]:
#Bag of Words for Harris/HOF_HOG
bow_train_HOF_HOG_Harris, bow_test_HOF_HOG_Harris = bag_of_words(desc_HOF_HOG_Harris_train, desc_HOF_HOG_Harris_test, 400)


In [ ]:
# Gabor/HOG descriptors for training videos
interest_Gabor_points_training = []
desc_HOG_Gabor_train = []
for Input in training_videos:
    interest_points = Gabor_points(Input, sigma, tau, N)
    interest_Gabor_points_training.append(interest_points)
    desc = HOG(Input, interest_points, 9, n, m)
    desc_HOG_Gabor_train.append(desc)


In [ ]:
interest_Gabor_points_test = []
desc_HOG_Gabor_test = []
for Input in test_videos:
    interest_points = Gabor_points(Input, sigma, tau, N)
    interest_Gabor_points_test.append(interest_points)
    desc = HOG(Input, interest_points, 9, n, m)
    desc_HOG_Gabor_test.append(desc)

    

In [ ]:
#Bag of Words for Gabor/HOG
bow_train_HOG_Gabor, bow_test_HOG_Gabor = bag_of_words(desc_HOG_Gabor_train, desc_HOG_Gabor_test, 400)



In [ ]:
desc_HOF_Gabor_train = []
i = 0
for Input in training_videos:
    desc = HOF(Input, interest_Gabor_points_training[i], 9, n, m)
    desc_HOF_Gabor_train.append(desc)
    i += 1

i = 0
desc_HOF_Gabor_test = []
for Input in test_videos:
    desc = HOF(Input, interest_Gabor_points_test[i], 9, n, m)
    desc_HOF_Gabor_test.append(desc)
    i += 1


In [ ]:
#Bag of Words for Gabor/HOF
bow_train_HOF_Gabor, bow_test_HOF_Gabor = bag_of_words(desc_HOF_Gabor_train, desc_HOF_Gabor_test, 20)


In [ ]:
desc_HOF_HOG_Gabor_train = []
i = 0
for Input in training_videos:
    desc = HOF_HOG(Input, interest_Gabor_points_training[i], 9, n, m)
    desc_HOF_HOG_Gabor_train.append(desc)
    i += 1

i = 0
desc_HOF_HOG_Gabor_test = []
for Input in test_videos:
    desc = HOF_HOG(Input, interest_Gabor_points_test[i], 9, n, m)
    desc_HOF_HOG_Gabor_test.append(desc)
    i += 1


In [ ]:
bow_train_HOF_HOG_Gabor, bow_test_HOF_HOG_Gabor = bag_of_words(desc_HOF_HOG_Gabor_train, desc_HOF_HOG_Gabor_test, 400)


In [ ]:
combo = 'detector:\tHarris \ndescriptor:\tHOF_HOG_Gabor'
bow_train = bow_train_HOG_Harris
bow_test = bow_test_HOG_Harris
accuracy, pred = svm_train_test(bow_train, training_labels, bow_test, test_labels)
print('\033[1m' + combo + '\033[0m')
print()
print('The accuracy for the combination is ' + str(format(accuracy*100, ".3f")) + "%")
print()
print("Here is a table with detailed results:")
print()
k = test_videos_name
a = test_labels
b = pred
c = a == b
print("Video Name \t \t \t Correct Prediction")

for i in range(len(a)):
    print(k[i] + '\t ' + a[i] + '\t ' + b[i]+ '\t ' + str(c[i]))


In [ ]:
combo = 'detector:\tHarris \ndescriptor:\tHOF'
bow_train = bow_train_HOF_Harris
bow_test = bow_test_HOF_Harris
accuracy, pred = svm_train_test(bow_train, training_labels, bow_test, test_labels)
print('\033[1m' + combo + '\033[0m')
print()
print('The accuracy for the combination is ' + str(format(accuracy*100, ".3f")) + "%")
print()
print("Here is a table with detailed results:")
print()
k = test_videos_name
a = test_labels
b = pred
c = a == b
print("Video Name \t \t \t Correct Prediction")

for i in range(len(a)):
    print(k[i] + '\t ' + a[i] + '\t ' + b[i]+ '\t ' + str(c[i]))


In [ ]:
combo = 'detector:\tHarris \ndescriptor:\tHOF and HOG'
bow_train = bow_train_HOF_HOG_Harris
bow_test = bow_test_HOF_HOG_Harris
accuracy, pred = svm_train_test(bow_train, training_labels, bow_test, test_labels)
print('\033[1m' + combo + '\033[0m')
print()
print('The accuracy for the combination is ' + str(format(accuracy*100, ".3f")) + "%")
print()
print("Here is a table with detailed results:")
print()
k = test_videos_name
a = test_labels
b = pred
c = a == b
print("Video Name \t \t \t Correct Prediction")

for i in range(len(a)):
    print(k[i] + '\t ' + a[i] + '\t ' + b[i]+ '\t ' + str(c[i]))


In [ ]:
combo = 'detector:\tGabor \ndescriptor:\tHOG'
bow_train = bow_train_HOG_Gabor
bow_test = bow_test_HOG_Gabor
accuracy, pred = svm_train_test(bow_train, training_labels, bow_test, test_labels)
print('\033[1m' + combo + '\033[0m')
print()
print('The accuracy for the combination is ' + str(format(accuracy*100, ".3f")) + "%")
print()
print("Here is a table with detailed results:")
print()
k = test_videos_name
a = test_labels
b = pred
c = a == b
print("Video Name \t \t \t Correct Prediction")

for i in range(len(a)):
    print(k[i] + '\t ' + a[i] + '\t ' + b[i]+ '\t ' + str(c[i]))


In [ ]:
combo = 'detector:\tGabor \ndescriptor:\tHOF'
bow_train = bow_train_HOF_Gabor
bow_test = bow_test_HOF_Gabor
accuracy, pred = svm_train_test(bow_train, training_labels, bow_test, test_labels)
print('\033[1m' + combo + '\033[0m')
print()
print('The accuracy for the combination is ' + str(format(accuracy*100, ".3f")) + "%")
print()
print("Here is a table with detailed results:")
print()
k = test_videos_name
a = test_labels
b = pred
c = a == b
print("Video Name \t \t \t Correct Prediction")

for i in range(len(a)):
    print(k[i] + '\t ' + a[i] + '\t ' + b[i]+ '\t ' + str(c[i]))


In [3]:
combo = 'detector:\tGabor \ndescriptor:\tHOF and HOG'
bow_train = bow_train_HOF_HOG_Gabor
bow_test = bow_test_HOF_HOG_Gabor
accuracy, pred = svm_train_test(bow_train, training_labels, bow_test, test_labels)
print('\033[1m' + combo + '\033[0m')
print()
print('The accuracy for the combination is ' + str(format(accuracy*100, ".3f")) + "%")
print()
print("Here is a table with detailed results:")
print()
k = test_videos_name
a = test_labels
b = pred
c = a == b
print("Video Name \t \t \t Correct Prediction")

for i in range(len(a)):
    print(k[i] + '\t ' + a[i] + '\t ' + b[i]+ '\t ' + str(c[i]))


NameError: name 'bow_train_HOF_HOG_Gabor' is not defined